### Creative Solution More Training - for more information see CreativeSolution.md

##### Import and install Dependencies

In [ ]:
! pip install scikit-image opencv-python

In [1]:
import cv2
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

C:\Users\haley\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the data

In [2]:
# Load dataset
X = np.load("Xset.npy")  # Shape: (num_samples, height, width, channels)
Y = np.load("Yset.npy")  # Shape: (num_samples,)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#### LBP Features

In [3]:
def extract_lbp_features(image, P=8, R=1):
    """Extract LBP features from a single grayscale image."""
    lbp = local_binary_pattern(image, P, R, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize histogram
    return hist

# Extract LBP for all images
lbp_features = np.array([extract_lbp_features(img) for img in X_train])

#### Prepare data to train MobileNetV2 model

In [4]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1).repeat(1, 3, 1, 1) / 255.0  # Normalize & Convert to 3 channels
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
Y_train_tensor = torch.argmax(Y_train_tensor, dim=1)

mn_train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)  

mn_train_loader = DataLoader(mn_train_dataset, batch_size=32, shuffle=True)

#### Create and Train (or Load) MobileNet Model

In [6]:
loading_model = True # SET BY USER
model = models.mobilenet_v2(pretrained=True)  # Load pretrained MobileNetV2

model.classifier = nn.Identity()  # Remove the last layer (classifier)

if loading_model:
    # Load the saved state dictionary
    model.load_state_dict(torch.load('mobilenet_v2_model.pth'))

    model.eval()
    print("Loaded Model!")

else:
    model.eval()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in mn_train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)  
            loss = criterion(outputs, labels)
            loss.backward()  
            optimizer.step() 
            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(mn_train_loader)}")


Loaded Model!


C:\Users\haley\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\haley\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#### Get embeddings from model

In [13]:
loading_embeddings = True

if loading_embeddings:
    mobilenet_features = np.load("embeddings_mobilenet_feat_ext.npy")

else:
    def extract_embeddings(model: nn.Module, x_train: torch.Tensor, batch_size: int = 32) -> torch.Tensor:
        embeddings = []
        model.eval()
        
        with torch.no_grad():  # We don't need gradients for embedding extraction
            for i in range(0, len(x_train), batch_size):
                inputs_batch = x_train[i:i+batch_size]
            # Move to GPU if necessary
                
                batch_embeddings = model(inputs_batch)  # Forward pass through the model
                embeddings.append(batch_embeddings.cpu())  # Move back to CPU if on GPU

        return torch.cat(embeddings, dim=0)  # Concatenate all the embeddings into a single tensor


    embeddings = extract_embeddings(model, X_train_tensor)
    mobilenet_features = embeddings.cpu().numpy() 
    np.save("embeddings_mobilenet_feat_ext.npy", mobilenet_features)

print(mobilenet_features.shape)

(6400, 1280)


#### Combined Features

In [14]:
# Combine features
X_combined = np.hstack((lbp_features, mobilenet_features))

# Save combined features
np.save("combined_features.npy", X_combined)

##### Train Random Forest Classifier on combined X

In [15]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y_train, test_size=0.2, random_state=42)

# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Accuracy: 0.9930
